Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Azure Machine Learning Pipeline with AutoMLStep
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

In [ ]:
# SDK バージョンを確認します。定期的に pip install --upgrade azureml-core を実行し、最新に保つことを推奨します。

import azureml.core
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

In [ ]:
# ワークスペース情報を取得します。

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment

In [ ]:
# 実験名を定義します。

from azureml.core.experiment import Experiment

experiment_name = 'pipeline-cycle-test'

experiment = Experiment(ws, experiment_name)
experiment

## Create or Attach an AmlCompute cluster

In [ ]:
# パイプラインの作成、実行するコンピューティング クラスターを作成します。既に存在する場合は作成せずスキップします。

from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## Prepare DataSet

In [ ]:
# ストレージ アカウント上のファイルを読み込んでデータセットを作成します。
# ラベル列はも併せて指定します。

from azureml.core import Datastore
from azureml.core.dataset import Dataset

datastore = ws.get_default_datastore()
dataset = Dataset.Tabular.from_delimited_files(path=(datastore,'Datasets/machineData.csv'))
label_column = 'ERP'

In [ ]:
# データセットの内容を一部表示します。

dataset.to_pandas_dataframe().info()
dataset.to_pandas_dataframe().head()

## Define AutoML Step

In [ ]:
# 自動機械学習用ステップの実行条件を AutMLConfig で定義します。

from azureml.train.automl import AutoMLConfig

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'r2_score',
    "enable_early_stopping": True, 
    "experiment_timeout_hours": 0.3,
    "iterations": 1,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1
}

automl_config = AutoMLConfig(task = 'regression',
                             allowed_models=['XGBoostRegressor'],
                             compute_target = compute_target,
                             training_data = dataset,
                             label_column_name = label_column,
                             enable_voting_ensemble=False,
                             enable_stack_ensemble=False,
                             **automl_settings
)

In [ ]:
# 後続のステップに自動機械学習結果 (Metrics) を引き継ぐよう、PipeLineData を定義します。

from azureml.pipeline.core import TrainingOutput, PipelineData

metrics_data = PipelineData(name='metrics_data',
                            datastore=datastore,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

In [ ]:
# AutMLConfig と PipeLineData を指定して自動機械学習用ステップ AutoMLStep を定義します。

from azureml.pipeline.steps import AutoMLStep

automl_step = AutoMLStep(
    name='automl_regression',
    automl_config=automl_config,
    enable_default_model_output=False,
    enable_default_metrics_output=False,
    outputs=[metrics_data],
    allow_reuse=False)

## Define Register ACI Endpoint Step

In [ ]:
%%writefile register_model.py
# モデルを登録するスクリプトです。
from azureml.core import Workspace, Experiment ,Environment
from azureml.core.model import Model, Dataset, InferenceConfig
from azureml.core.run import Run, _OfflineRun
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException
import json
import argparse

# 引数の読み込み
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", required=True)
parser.add_argument("--metrics_data", required=True)
parser.add_argument("--primary_metric", required=True)
parser.add_argument("--aciservice_name", required=True)
args = parser.parse_args()

print(f"model_name : {args.model_name}")
print(f"metrics_data : {args.metrics_data}")
print(f"primary_metric : {args.primary_metric}")
print(f"aciservice_name : {args.aciservice_name}")

run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

# 自動機械学習結果の読み込んでプライマリ メトリックの最大または最小値を取得
with open(args.metrics_data) as f:
    j = json.load(f)

def key_func(n):
    return n[args.primary_metric]

if args.primary_metric == "r2_score":
    a = max(j.values(), key=key_func)[args.primary_metric]
else:
    a = min(j.values(), key=key_func)[args.primary_metric]

# 上述で取得したプライマリ メトリックの値を持つ実行 ID を特定
count = 0
best_runid = ""
while count < len(j):
    b = list(j.values())[count][args.primary_metric]
    c = list(j.keys())[count]
    if a == b:
        best_runid = c
    count += 1

print(f"best_runid : {best_runid}")

experiment = Experiment(workspace=ws, name="automl_test_model")

best_run = Run(experiment=experiment, run_id=best_runid)

# 特定した実行 ID で生成されたモデル ファイル、依存関係ファイル、エントリ スクリプトをダウンロードしてモデル登録
best_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', './env.yml')
best_run.download_file('outputs/model.pkl', './model.pkl')

model = Model.register(workspace=ws,
                       model_path="./model.pkl",
                       model_name=args.model_name)

print("Registered version {0} of model {1}".format(model.version, model.name))

environment = Environment.from_conda_specification('automlenv', file_path="./env.yml")

# 既に同じ名前の ACI Web エンドポイントが存在する場合には更新、存在しない場合にはエンドポイントを作成
try:
    service = AciWebservice(name=args.aciservice_name, workspace=ws)
    print('Found existing service, update it.')
    inference_config = InferenceConfig(entry_script='./score.py', environment=environment)
    service.update(models=[model], inference_config=inference_config)

except WebserviceException:
    inference_config = InferenceConfig(entry_script='./score.py', environment=environment)
    aci_config = AciWebservice.deploy_configuration(cpu_cores=1.8, memory_gb=4)
    service = Model.deploy(workspace=ws,
                           name=args.aciservice_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aci_config,
                           overwrite=True)

service.wait_for_deployment(show_output=True)

print(service.state)

In [ ]:
# モデル登録ステップを定義します。
# PipelineParameter としてモデル名、プライマリ メトリック、ACI Web エンドポイントの名前をパイプライン実行と毎に指定できるようにします。

from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

aml_run_config = RunConfiguration()

model_name = PipelineParameter("model_name", default_value="automlmodel")
primary_metric = PipelineParameter("primary_metric", default_value="r2_score")
aciservice_name = PipelineParameter("aciservice_name", default_value="aciservice")

register_step = PythonScriptStep(script_name="register_model.py",
                                 name="register_model",
                                 allow_reuse=True,
                                 arguments=[
                                    "--model_name", model_name,
                                    "--metrics_data", metrics_data,
                                    "--primary_metric", primary_metric,
                                    "--aciservice_name", aciservice_name
                                 ],
                                 inputs=[metrics_data],
                                 compute_target=compute_target,
                                 runconfig=aml_run_config)

## Create Pipeline

In [ ]:
# パイプラインを作成します。

from azureml.pipeline.core import Pipeline

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step, register_step])

## Run

In [ ]:
# 作成したパイプラインを実行します。

pipeline_run = experiment.submit(pipeline)

In [ ]:
# 実行結果をモニターします。

from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
# パイプライン実行の完了を待ちます。

pipeline_run.wait_for_completion()